<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 2
#### Student Name: Deshui Yu      Liangjing Yang
#### Student ID: 34253599      34060871

Date: 28/09/2024

    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Patent Files](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Whatever else](#latin) <br>
[5. Writing to CSV/JSON File](#write) <br>
$\;\;\;\;$[5.1. Verification - using the sample files](#test_xml) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

This project involves cleansing and analyzing a retail transactional dataset from DigiCO, an online electronics store in Melbourne. The task is to detect and fix errors, impute missing values, and remove outliers using exploratory data analysis (EDA). Cleaned data will be saved in the required output files, and the process will be documented in the final report.

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **re:** to define and use regular expressions
* **pandas:** to manage and analyze data.
* **datetime** to handle dates and times.

In [69]:
import pandas as pd
import numpy as np
import math
import ast
from collections import Counter
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk


<div class="alert alert-block alert-warning">

## 3.  Examining Raw Data <a class="anchor" name="examine"></a>

 </div>

我们可以看到这三个文件都包含以下数据列：order_id、customer_id、date、nearest_warehouse、shopping_cart、order_price、delivery_charges、customer_lat、customer_long、coupon_discount、order_total、season、is_expedited_delivery、distance_to_nearest_warehouse、latest_customer_review 和 is_happy_customer。其中，coupon_discount、delivery_charges、shopping_cart 中的商品数量、order_id、customer_id 和 latest_customer_review 是没有错误的数据。在 Group181_missing_data.csv 文件中，is_happy_customer 列的数据缺失，其数据类型为 float64；在 Group181_dirty_data.csv 中存在错误数据；而在 Group181_outlier_data.csv 文件中则有异常数据。
通过数据逻辑我们可以发现，date和season数据是有关系的，customer_lat and customer_long 和 distance_to_nearest_warehouse

In [70]:
dirty_file_path = 'Group181_dirty_data.csv'
dirty_data = pd.read_csv(dirty_file_path)
dirty_data['error'] = 0
# dirty_data.info()

In [71]:
missing_file_path = 'Group181_missing_data.csv'
missing_data = pd.read_csv(missing_file_path)


In [72]:
outlier_file_path = 'Group181_outlier_data.csv'
outlier_data = pd.read_csv(outlier_file_path)

<div class="alert alert-block alert-warning"> 

## 4.  Detect and fix errors in dirty_data <a class="anchor" name="load"></a>

</div>

<div class="alert alert-block alert-info">
    
### 4.1. Fix the date <a class="anchor" name="Reg_Exp"></a>

Through examining the current data, the issue was identified as the date values, which were supposed to be in the YYYY-MM-DD format, mistakenly being formatted as YYYY-DD-MM and DD-MM-YYYY. Additionally, based on logical reasoning, the date and season are highly correlated, so after fixing the date data, the season data should also be corrected accordingly.

In [73]:
# Check and mark invalid dates
# reference from chatGPT
# Temporarily convert the 'date' column to datetime format, marking invalid dates as NaT (Not a Time)
temp_dates = pd.to_datetime(dirty_data['date'], errors='coerce')
# Find the invalid date
invalid_dates_temp = dirty_data[temp_dates.isna()]
# print(invalid_dates_temp["date"])

In [74]:
# 将 date 列转换为 datetime，错误值会变成 NaT
temp_dates = pd.to_datetime(dirty_data['date'], errors='coerce')

# 将 NaT 行标记为 1，表示有错误
dirty_data.loc[temp_dates.isna(), 'error'] = 1

# 获取 dirty_data 中 error 列标记为 1 的行号
error_indices = dirty_data.index[dirty_data['error'] == 1]

# 打印出这些行号
print(error_indices)

# 准备修复日期并标记修复
fixed_dates = []
for idx, date_str in enumerate(dirty_data['date'].astype(str)):
    parts = date_str.split('-')
    # Check if format is DD-MM-YYYY and fix to YYYY-MM-DD
    if len(parts) == 3 and int(parts[0]) <= 12 and int(parts[1]) <= 31:
        fixed_dates.append(f'{parts[2]}-{parts[1]}-{parts[0]}')  # DD-MM-YYYY -> YYYY-MM-DD
        dirty_data.loc[idx, 'error'] = 2  # 只标记当前行
    # Check if format is YYYY-DD-MM and fix to YYYY-MM-DD
    elif len(parts) == 3 and int(parts[1]) > 12 and int(parts[2]) <= 12:
        fixed_dates.append(f'{parts[0]}-{parts[2]}-{parts[1]}')  # YYYY-DD-MM -> YYYY-MM-DD
        dirty_data.loc[idx, 'error'] = 2  # 只标记当前行
    # Keep original format for other cases
    else:
        fixed_dates.append(date_str)

# 将修复后的日期转换回 datetime 格式，并替换原始的 'date' 列
dirty_data['date'] = pd.to_datetime(fixed_dates, errors='coerce')


Int64Index([33, 43, 69, 110, 172, 240, 246, 285, 291, 371, 443, 466, 467], dtype='int64')


<div class="alert alert-block alert-info">
    
### 4.2. Fix the season <a class="anchor" name="Reg_Exp"></a>

Since we know that the date and season data are logically related, once the date data has been corrected, the season data should also be updated accordingly.

In [75]:
dirty_data['date'] = pd.to_datetime(dirty_data['date'], errors='coerce')

# 定义获取季节的函数
def get_correct_season(month):
    if month in [12, 1, 2]:
        return 'Summer'
    elif month in [3, 4, 5]:
        return 'Autumn'
    elif month in [6, 7, 8]:
        return 'Winter'
    elif month in [9, 10, 11]:
        return 'Spring'

# 第一步：标记错误的 season 数据

# 提取没有标记 error 的数据行
row_season_data = dirty_data[dirty_data['error'] == 0].copy()

# 提取月份
row_season_data['month'] = row_season_data['date'].dt.month

# 应用获取正确季节的函数
row_season_data['correct_season'] = row_season_data['month'].apply(get_correct_season)

# 检查 season 列是否正确
row_season_data['season_is_wrong'] = row_season_data['season'] != row_season_data['correct_season']

# 获取 season 错误的行的索引
wrong_season_indices = row_season_data.index[row_season_data['season_is_wrong']]

# 将对应的行在 dirty_data 中标记为 error
dirty_data.loc[wrong_season_indices, 'error'] = 1

# 删除临时列，并忽略不存在的列错误
dirty_data.drop(columns=['month', 'correct_season', 'season_is_wrong'], inplace=True, errors='ignore')


# 获取 dirty_data 中 error 列标记为 1 的行号
error_indices = dirty_data.index[dirty_data['error'] == 1]

# 打印出这些行号
print(error_indices)

Int64Index([  8,  24,  57,  60,  66,  95, 100, 115, 116, 132, 138, 147, 195,
            199, 211, 212, 213, 222, 245, 250, 255, 267, 323, 446, 462, 468,
            496],
           dtype='int64')


In [76]:
fixed_seasons = []

# 遍历每一行，修正季节数据
for idx, date in dirty_data.iterrows():
    month = date['date'].month  # 提取月份
    # 如果 error 列为 1，则修正季节
    if dirty_data.loc[idx, 'error'] == 1:
        if month in [9, 10, 11]:
            fixed_seasons.append('Spring')
        elif month in [12, 1, 2]:
            fixed_seasons.append('Summer')
        elif month in [3, 4, 5]:
            fixed_seasons.append('Autumn')
        elif month in [6, 7, 8]:
            fixed_seasons.append('Winter')
        # 将错误标记修正为 2
        dirty_data.loc[idx, 'error'] = 2
    else:
        # 如果 error 不为 1，则保留原来的 season 值
        fixed_seasons.append(dirty_data.loc[idx, 'season'])

# 将修复后的 season 列替换掉原始的 season 列
dirty_data['season'] = fixed_seasons

# 打印修复后的 date 和 season 列
print(dirty_data[['date', 'season']])

          date  season
0   2019-01-23  Summer
1   2019-11-07  Spring
2   2019-01-14  Summer
3   2019-10-31  Spring
4   2019-04-02  Autumn
..         ...     ...
495 2019-11-03  Spring
496 2019-02-07  Summer
497 2019-05-29  Autumn
498 2019-05-03  Autumn
499 2019-09-06  Spring

[500 rows x 2 columns]


<div class="alert alert-block alert-info">
    
### 4.3. Fix the customer_lat, customer_long<a class="anchor" name="Reg_Exp"></a>

In [77]:
for idx, row in dirty_data.iterrows():
    lat_issue = row['customer_lat'] > 0  # 纬度应为负值
    long_issue = row['customer_long'] < 0  # 经度应为正值

    # 如果存在纬度问题或经度问题，则将 error 列标记为 1
    if lat_issue or long_issue:
        dirty_data.loc[idx, 'error'] = 1
        
# 获取 dirty_data 中 error 列标记为 1 的行号
error_indices = dirty_data.index[dirty_data['error'] == 1]

# 打印出这些行号
print(error_indices)


Int64Index([ 48,  68,  77,  87,  93, 136, 137, 140, 187, 196, 198, 216, 220,
            230, 242, 297, 299, 361, 366, 393, 408, 427, 438, 452, 464, 469,
            495],
           dtype='int64')


In [78]:
# 第二步：修正标记为 error == 1 的行（交换错误的纬度和经度）
for idx, row in dirty_data.iterrows():
    # 如果该行的 error 列标记为 1，则执行修正操作
    if dirty_data.loc[idx, 'error'] == 1:
        # 获取条件：纬度为正或经度为负
        condition = (dirty_data.loc[idx, 'customer_long'] < 0) or (dirty_data.loc[idx, 'customer_lat'] > 0)
        
        if condition:
            # 交换 customer_lat 和 customer_long 的值
            temp_lat = dirty_data.loc[idx, 'customer_lat']
            dirty_data.loc[idx, 'customer_lat'] = dirty_data.loc[idx, 'customer_long']
            dirty_data.loc[idx, 'customer_long'] = temp_lat
            
            # 将该行的 error 更新为 2，表示已修正
            dirty_data.loc[idx, 'error'] = 2

<div class="alert alert-block alert-info">
    
### 4.4. Fix the nearest_warehouse<a class="anchor" name="Reg_Exp"></a>

In [79]:
warehouse_counts = dirty_data['nearest_warehouse'].value_counts()
print(warehouse_counts)

for idx, row in dirty_data.iterrows():
    # 如果 error 列标记为 0，检查 nearest_warehouse
    if dirty_data.loc[idx, 'error'] == 0:
        # 如果 nearest_warehouse 不在指定的三个仓库列表中，标记为 error = 1
        if dirty_data.loc[idx, 'nearest_warehouse'] not in ["Nickolson", "Thompson", "Bakers"]:
            dirty_data.loc[idx, 'error'] = 1

Thompson     187
Nickolson    186
Bakers       108
bakers         9
thompson       7
nickolson      3
Name: nearest_warehouse, dtype: int64


In [80]:
# 遍历 dirty_data 中的每一行
for idx, row in dirty_data.iterrows():
    # 如果 error 列标记为 1，检查 nearest_warehouse
    if dirty_data.loc[idx, 'error'] == 1:
        # 获取当前 nearest_warehouse 的值
        warehouse = dirty_data.loc[idx, 'nearest_warehouse'].lower()  # 将其转为小写，便于比较
        
        # 如果 warehouse 是 "bakers"、"thompson" 或 "nickolson"，则将其转为首字母大写
        if warehouse in ["bakers", "thompson", "nickolson"]:
            dirty_data.loc[idx, 'nearest_warehouse'] = warehouse.capitalize()  # 首字母大写
            dirty_data.loc[idx, 'error'] = 2  # 将 error 列标记为 2

<div class="alert alert-block alert-info">
    
### 4.5. Fix the distance_to_nearest_warehouse<a class="anchor" name="Reg_Exp"></a>

In [81]:
# 读取仓库数据
warehouse = pd.read_csv("warehouses.csv")

# 创建字典，将仓库名称与对应的纬度和经度配对
lat = dict(zip(warehouse['names'], warehouse['lat']))
lon = dict(zip(warehouse['names'], warehouse['lon']))

# 定义哈弗赛因公式计算两点之间的距离
def haversine(lat1, lon1, lat2, lon2):
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
    lat1 = lat1 * math.pi / 180.0
    lat2 = lat2 * math.pi / 180.0

    a = (pow(math.sin(dLat / 2), 2) + 
         pow(math.sin(dLon / 2), 2) * 
         math.cos(lat1) * math.cos(lat2))

    # 地球半径（单位：公里）
    rad = 6378
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

# 初始化 distance_computed 列
dirty_data['distance_computed'] = None

# 遍历每个客户的记录，计算到 nearest_warehouse 的距离
for index, row in dirty_data.iterrows():
    # 获取客户的纬度和经度
    customer_lat = row['customer_lat']
    customer_long = row['customer_long']
    
    # 获取 nearest_warehouse 名称
    nearest_warehouse = row['nearest_warehouse']
    
    # 检查 nearest_warehouse 是否在字典中
    if nearest_warehouse in lat and nearest_warehouse in lon:
        # 计算客户与 nearest_warehouse 的距离
        dist = round(haversine(customer_lat, customer_long, lat[nearest_warehouse], lon[nearest_warehouse]), 4)
        
        # 将计算的距离存入 distance_computed 列
        dirty_data.loc[index, 'distance_computed'] = dist
        

In [82]:
# 初始化一个列表来记录不相等的行的行号
mismatch_indices = []

# 遍历 dirty_data 的每一行
for idx, row in dirty_data.iterrows():
    # 只对 error 为 0 的行进行操作
    if dirty_data.loc[idx, 'error'] == 0:
        # 获取原始的 distance_to_nearest_warehouse 和 新计算的 distance_computed
        original_distance = dirty_data.loc[idx, 'distance_to_nearest_warehouse']
        computed_distance = dirty_data.loc[idx, 'distance_computed']
        
        # 如果两者不相等
        if original_distance != computed_distance:
            mismatch_indices.append(idx)
            dirty_data.loc[idx, 'distance_to_nearest_warehouse'] = computed_distance
            dirty_data.loc[idx, 'error'] = 2

# 打印记录不相等的行号
print(f"以下行的距离值不相等，需要修正：{mismatch_indices}")

# 删除 distance_computed 列
dirty_data.drop(columns=['distance_computed'], inplace=True)

# 打印修正后的数据进行检查
print(dirty_data.loc[mismatch_indices, ['distance_to_nearest_warehouse', 'error']])

以下行的距离值不相等，需要修正：[5, 18, 19, 29, 34, 39, 76, 78, 88, 96, 99, 112, 124, 143, 162, 165, 174, 207, 254, 274, 279, 311, 324, 346, 352, 356, 381, 385, 389, 396, 400, 401, 439, 440, 484]
     distance_to_nearest_warehouse  error
5                           3.4051      2
18                          1.3150      2
19                          0.7133      2
29                          0.7403      2
34                          1.5612      2
39                          0.9221      2
76                          2.1753      2
78                          5.5590      2
88                          0.7450      2
96                          0.9259      2
99                          2.1706      2
112                         5.0801      2
124                         0.9636      2
143                         0.6619      2
162                         1.5315      2
165                         1.7155      2
174                         0.9547      2
207                         0.8881      2
254                   

In [43]:
# 获取 dirty_data 中 error 列标记为 1 的行号
error_indices = dirty_data.index[dirty_data['error'] == 0]

# 打印出这些行号
print(error_indices)

Int64Index([  0,   1,   2,   3,   4,   6,   7,   9,  10,  11,
            ...
            487, 488, 489, 490, 491, 492, 494, 497, 498, 499],
           dtype='int64', length=365)


<div class="alert alert-block alert-info">
    
### 4.6. Fix the order_total<a class="anchor" name="Reg_Exp"></a>

In [83]:
# 初始化一个 Counter 来统计每个品牌的销售数量
brand_counter = Counter()

# 遍历 outlier_data 数据集以获取所有品牌
for index, row in outlier_data.iterrows():
    # 将 shopping_cart 列的字符串转换为 Python 列表
    shopping_cart = ast.literal_eval(row['shopping_cart'])
    # 遍历购物车中的每个商品
    for item in shopping_cart:
        brand_name = item[0]  # 获取商品的品牌名称
        brand_counter[brand_name] += 1  # 统计每个品牌的出现次数

# 获取所有的品牌列表
item_types = list(brand_counter.keys())
print(item_types)

# 初始化矩阵 A 和向量 b
A = np.zeros((len(outlier_data), len(item_types)))  # A 矩阵的形状是 (订单数量, 品牌数量)
b = np.zeros(len(outlier_data))  # b 是存储订单价格的向量

# 遍历 outlier_data 数据集来填充矩阵 A 和向量 b
for index, row in outlier_data.iterrows():
    # 将 shopping_cart 列的字符串形式转换为 Python 列表
    shopping_cart = ast.literal_eval(row["shopping_cart"])  # 使用 ast.literal_eval 提高安全性
    # 将订单的价格存储到向量 b 中
    b[index] = row["order_price"]
    # 遍历购物车中的每个商品和数量
    for item in shopping_cart:
        brand_name = item[0]
        quantity = item[1]
        # 如果商品属于已知的 item_types
        if brand_name in item_types:
            # 找到该商品在 item_types 列表中的索引
            item_index = item_types.index(brand_name)
            # 将商品的数量添加到矩阵 A 的相应位置
            A[index, item_index] += quantity

# 检查 A 和 b 中是否有全为零的行或无效值
valid_indices = np.where(A.any(axis=1) & ~np.isnan(b))[0]
A = A[valid_indices]
b = b[valid_indices]

# 使用 np.linalg.lstsq() 计算每个品牌的价格
prices, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

# 对价格进行四舍五入处理，不保留小数点
rounded_prices = np.round(prices)

# 将结果转换为品牌价格的字典
price_dict = dict(zip(item_types, rounded_prices))

# 输出每个品牌的价格
print("Brand Prices (Rounded):")
for brand, price in price_dict.items():
    print(f"Brand: {brand}, Price: {int(price)}")  # 转换为整数形式输出
    

['iAssist Line', 'Lucent 330S', 'Toshika 750', 'Thunder line', 'Olivia x460', 'Universe Note', 'iStream', 'Alcon 10', 'Candle Inferno', 'pearTV']
Brand Prices (Rounded):
Brand: iAssist Line, Price: 2225
Brand: Lucent 330S, Price: 1230
Brand: Toshika 750, Price: 4320
Brand: Thunder line, Price: 2180
Brand: Olivia x460, Price: 1225
Brand: Universe Note, Price: 3450
Brand: iStream, Price: 150
Brand: Alcon 10, Price: 8950
Brand: Candle Inferno, Price: 430
Brand: pearTV, Price: 6310


In [84]:
dirty_data['order_computed'] = None
# 遍历每一行数据
for idx, row in dirty_data.iterrows():
    # 将 shopping_cart 列的字符串转换为 Python 列表
    shopping_cart = ast.literal_eval(row['shopping_cart'])
    # 如果 error 列为 0，则计算购物车的总价
    if dirty_data.loc[idx, 'error'] == 0:
        total_price = 0
        # 遍历购物车中的每个商品
        for item in shopping_cart:
            brand, quantity = item  # 每个 item 是 (brand, quantity) 的形式
            # 计算当前商品的总价
            if brand in price_dict:
                total_price += price_dict[brand] * quantity
        # 将计算出来的总价存入 order_computed 列
        dirty_data.loc[idx, 'order_computed'] = total_price
    else:
        # 如果 error 不为 0，则将 order_computed 设置为 order_price
        dirty_data.loc[idx, 'order_computed'] = row['order_price']
# print(dirty_data['order_computed'])

0       8130.0
1       2750.0
2       6820.0
3       8555.0
4       6780.0
        ...   
495       6735
496      26035
497    30475.0
498     5905.0
499    21570.0
Name: order_computed, Length: 500, dtype: object


In [89]:
mismatch_rows = dirty_data[dirty_data['order_computed'] != dirty_data['order_price']]
mismatch_indices = mismatch_rows.index.tolist()
for idx in mismatch_indices:
    if dirty_data.loc[idx, 'error'] == 0:
        dirty_data.loc[idx, 'error'] = 1

import itertools
# 初始化一个列表来存储匹配成功的行号
successful_matches = []

# 遍历所有 error == 1 的行
for idx, row in dirty_data[dirty_data['error'] == 1].iterrows():
    shopping_cart = ast.literal_eval(row['shopping_cart'])
    original_order_price = row['order_price']

    # 获取购物车中商品的数量，保持数量不变
    quantities = [item[1] for item in shopping_cart]  # 只取数量，不替换品牌

    # 获取所有品牌的排列组合，确保每个品牌只出现一次
    if len(quantities) <= len(price_dict):  # 确保品牌数量足够
        brand_combinations = itertools.permutations(price_dict.keys(), len(quantities))

        # 遍历所有可能的品牌组合
        for possible_combination in brand_combinations:
            temp_total = 0
            temp_shopping_cart = []

            # 遍历购物车中的数量，使用不同品牌组合
            for brand, quantity in zip(possible_combination, quantities):
                temp_total += price_dict[brand] * quantity
                temp_shopping_cart.append((brand, quantity))

            # 如果替换后的购物车价格与原始 order_price 匹配（允许误差在10以内）
            if abs(temp_total - original_order_price) <= 5:
                print(f"在第 {idx} 行发现匹配成功，替换后的购物车：{temp_shopping_cart}")
                
                # 更新 dirty_data 中的 shopping_cart 和 order_computed
                dirty_data.loc[idx, 'shopping_cart'] = str(temp_shopping_cart)  # 将更新后的购物车存储为字符串
                dirty_data.loc[idx, 'order_computed'] = temp_total
                dirty_data.loc[idx, 'error'] = 2  # 匹配成功，标记 error 为 3
                
                # 记录匹配成功的行号
                successful_matches.append(idx)
                break  # 一旦找到匹配，跳出品牌组合的循环

# 输出匹配成功的行号和数量
print(f"匹配成功的行号: {successful_matches}")
print(f"匹配成功的行数: {len(successful_matches)}")

在第 327 行发现匹配成功，替换后的购物车：[('Toshika 750', 2), ('Universe Note', 1), ('Candle Inferno', 1)]
匹配成功的行号: [327]
匹配成功的行数: 1


In [ ]:
for idx, row in dirty_data[dirty_data['error'] == 1].iterrows():
  

Empty DataFrame
Columns: [is_happy_customer]
Index: []


In [52]:
# 获取 dirty_data 中 error 列标记为 1 的行号
error_indices = dirty_data.index[dirty_data['error'] == 3]

# 打印出这些行号
print(error_indices)

Int64Index([], dtype='int64')


In [19]:
sia = SentimentIntensityAnalyzer()

missing_data = missing_data.copy()

missing_data['latest_customer_review'] = missing_data['latest_customer_review'].astype(str)

for index, row in missing_data.iterrows():
    review_text = row['latest_customer_review']
    polarity_score = sia.polarity_scores(review_text)['compound']
    
    if polarity_score >= 0.05:
        missing_data.loc[index, 'review_sentiment'] = 1
    else:
        missing_data.loc[index, 'review_sentiment'] = 0

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - '/Users/yudeshui/nltk_data'
    - '/Users/yudeshui/anaconda3/nltk_data'
    - '/Users/yudeshui/anaconda3/share/nltk_data'
    - '/Users/yudeshui/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


<div class="alert alert-block alert-info">
    
### 4.8. Fix the order_total<a class="anchor" name="Reg_Exp"></a>

In [18]:
# 初始化 total_computed 列
dirty_data['total_computed'] = None
# 遍历每一行数据
for idx, row in dirty_data.iterrows():
    order_price = row['order_price']
    
    # 如果 error 列为 0，计算总价
    if dirty_data.loc[idx, 'error'] == 0:
        delivery_charges = row['delivery_charges']
        coupon_discount = row['coupon_discount'] / 100  # 将百分比折扣转换为小数
        # 计算总价：order_price 先应用折扣，再加上运费
        total_computed = order_price * (1 - coupon_discount) + delivery_charges
        # 将计算的总价存入 total_computed 列
        dirty_data.loc[idx, 'total_computed'] = total_computed
    else:
        # 如果 error 不为 0，则将 total_computed 设置为 order_total
        dirty_data.loc[idx, 'total_computed'] = row['order_total']


In [19]:
mismatch_rows = dirty_data[dirty_data['total_computed'] != dirty_data['order_total']]

# 遍历这些不匹配的行，更新 order_total，并设置 error = 2
for idx in mismatch_rows.index:
    # 将计算的 total_computed 赋值给 order_total
    dirty_data.loc[idx, 'order_total'] = dirty_data.loc[idx, 'total_computed']
    
    # 将 error 列更新为 2
    dirty_data.loc[idx, 'error'] = 2

In [158]:
dirty_data['order_computed'] = None
# 遍历每一行数据
for idx, row in dirty_data.iterrows():
    # 将 shopping_cart 列的字符串转换为 Python 列表
    shopping_cart = ast.literal_eval(row['shopping_cart'])

    # 如果 error 列为 0，则计算购物车的总价
    if dirty_data.loc[idx, 'error'] == 0:
        total_price = 0
        # 遍历购物车中的每个商品
        for item in shopping_cart:
            brand, quantity = item  # 每个 item 是 (brand, quantity) 的形式
            # 计算当前商品的总价
            if brand in price_dict:
                total_price += price_dict[brand] * quantity
        # 将计算出来的总价存入 order_computed 列
        dirty_data.loc[idx, 'order_computed'] = total_price
    else:
        # 如果 error 不为 0，则将 order_computed 设置为 order_price
        dirty_data.loc[idx, 'order_computed'] = row['order_price']


In [157]:
# 初始化 total_computed 列
dirty_data['total_computed'] = None
# 遍历每一行数据
for idx, row in dirty_data.iterrows():
    order_price = row['order_price']
    
    # 如果 error 列为 0，计算总价
    if dirty_data.loc[idx, 'error'] == 0:
        delivery_charges = row['delivery_charges']
        coupon_discount = row['coupon_discount'] / 100  # 将百分比折扣转换为小数
        # 计算总价：order_price 先应用折扣，再加上运费
        total_computed = order_price * (1 - coupon_discount) + delivery_charges
        # 将计算的总价存入 total_computed 列
        dirty_data.loc[idx, 'total_computed'] = total_computed
    else:
        # 如果 error 不为 0，则将 total_computed 设置为 order_total
        dirty_data.loc[idx, 'total_computed'] = row['order_total']
# 打印结果检查
print(dirty_data[['order_price', 'coupon_discount', 'delivery_charges', 'total_computed', 'order_total']])

     order_price  coupon_discount  delivery_charges total_computed  \
0           8130               10             98.05        7415.05   
1           2750               10             80.36        2555.36   
2           6820               15             81.29        5878.29   
3           8555               25             82.87        6499.12   
4          10080                0             68.21       10148.21   
..           ...              ...               ...            ...   
495         6735               25             84.17        5135.42   
496        26035               10             71.44       23502.94   
497        30475                5             66.35        29017.6   
498         4110                5             47.18        3951.68   
499        12390                0             73.81       12463.81   

     order_total  
0        7415.05  
1        2555.36  
2        5878.29  
3        6499.12  
4       10148.21  
..           ...  
495      5135.42  
496    

In [156]:

dirty_data['export_total_computed'] = None

# 遍历每一行数据
for idx, row in dirty_data.iterrows():
    order_computed = row['order_computed']
    
    # 如果 error 列为 0，计算总价
    if dirty_data.loc[idx, 'error'] == 0:
        delivery_charges = row['delivery_charges']
        coupon_discount = row['coupon_discount'] / 100  # 将百分比折扣转换为小数
        
        # 计算总价：order_computed 应用折扣后再加上运费
        export_total_computed = order_computed * (1 - coupon_discount) + delivery_charges
        
        # 将计算的总价存入 export_total_computed 列
        dirty_data.loc[idx, 'export_total_computed'] = export_total_computed
    else:
        # 如果 error 不为 0，则将 export_total_computed 设置为 order_total
        dirty_data.loc[idx, 'export_total_computed'] = row['order_total']


In [161]:
# 找出 order_computed 和 order_price 不同，且 total_computed 和 order_total 相同的行
mismatch_rows = dirty_data[
    (dirty_data['order_computed'] != dirty_data['order_price']) & 
    (dirty_data['export_total_computed'] == dirty_data['order_total'])
]

# 获取这些行的行号
mismatch_indices = mismatch_rows.index.tolist()

# 计算不相等的行数
mismatch_count = len(mismatch_indices)

# 输出不相等的行数
print(f"order_computed 和 order_price 不同，且 total_computed 和 order_total 相同的行数: {mismatch_count}")

# 输出具体的行号
print(f"行号: {mismatch_indices}")

# 如果你想查看具体的不同的行，可以打印这些行
print(mismatch_rows[['order_computed', 'order_price', 'total_computed', 'order_total']])


order_computed 和 order_price 不同，且 total_computed 和 order_total 相同的行数: 26
行号: [11, 15, 16, 94, 126, 130, 159, 168, 181, 235, 241, 248, 256, 261, 280, 288, 327, 336, 370, 382, 391, 395, 426, 463, 475, 499]
    order_computed  order_price total_computed  order_total
11         13080.0         5060        5121.68     13141.68
15          5550.0         6820        5864.79      4785.29
16         20660.0        21260       21313.28     20713.28
94          2375.0        10080        9153.28      2218.78
126         3455.0        24550       20962.52      3031.77
130        33660.0        14690       14773.99     33743.99
159        12640.0        13090       13169.07     12719.07
168         1655.0        14845       14912.05      1722.05
181         6555.0         7170        6530.53      5977.03
235        14080.0        13210       11979.16     12762.16
241        33925.0        10230        8764.69     28905.44
248        13480.0         8125        7003.77     11555.52
256        23970

In [159]:
# 找出 order_computed 和 order_price 不同，且 export_total_computed, order_total 和 total_computed 相同的行
mismatch_rows = dirty_data[
    (dirty_data['order_computed'] != dirty_data['order_price']) & 
    # (dirty_data['export_total_computed'] == dirty_data['order_total']) & 
    (dirty_data['export_total_computed'] == dirty_data['total_computed'])
]

# 获取这些行的行号
mismatch_indices = mismatch_rows.index.tolist()

# 计算不相等的行数
mismatch_count = len(mismatch_indices)

# 输出不相等的行数
print(f"order_computed 和 order_price 不同，且 export_total_computed 和 order_total 相同的行数: {mismatch_count}")

# 输出具体的行号
print(f"行号: {mismatch_indices}")

# 如果你想查看具体的不同的行，可以打印这些行
print(mismatch_rows[['order_computed', 'order_price', 'export_total_computed', 'order_total', 'total_computed']])


order_computed 和 order_price 不同，且 export_total_computed 和 order_total 相同的行数: 0
行号: []
Empty DataFrame
Columns: [order_computed, order_price, export_total_computed, order_total, total_computed]
Index: []


In [137]:
# 初始化 total_computed 列
dirty_data['total_computed'] = None

# 遍历每一行数据
for idx, row in dirty_data.iterrows():
    order_price = row['order_price']
    
    # 如果 error 列为 0，计算总价
    if dirty_data.loc[idx, 'error'] == 0:
        delivery_charges = row['delivery_charges']
        coupon_discount = row['coupon_discount'] / 100  # 将百分比折扣转换为小数
        # 计算总价：order_price 先应用折扣，再加上运费
        total_computed = order_price * (1 - coupon_discount) + delivery_charges
        # 将计算的总价存入 total_computed 列
        dirty_data.loc[idx, 'total_computed'] = total_computed
    else:
        # 如果 error 不为 0，则将 total_computed 设置为 order_total
        dirty_data.loc[idx, 'total_computed'] = row['order_total']

# 打印结果检查
print(dirty_data[['order_price', 'coupon_discount', 'delivery_charges', 'total_computed', 'order_total']])
  


     order_price  coupon_discount  delivery_charges total_computed  \
0           8130               10             98.05        7415.05   
1           2750               10             80.36        2555.36   
2           6820               15             81.29        5878.29   
3           8555               25             82.87        6499.12   
4          10080                0             68.21       10148.21   
..           ...              ...               ...            ...   
495         6735               25             84.17        5135.42   
496        26035               10             71.44       23502.94   
497        30475                5             66.35        29017.6   
498         4110                5             47.18        3951.68   
499        12390                0             73.81       12463.81   

     order_total  
0        7415.05  
1        2555.36  
2        5878.29  
3        6499.12  
4       10148.21  
..           ...  
495      5135.42  
496    

In [140]:
# 找到 total_computed 和 order_total 不相等的行
mismatch_rows = dirty_data[abs(dirty_data['total_computed'] - dirty_data['order_total']) > 4]  # 允许少量的浮点误差

# 获取这些行的行号
mismatch_indices = mismatch_rows.index.tolist()

# 计算不相等的行数
mismatch_count = len(mismatch_indices)

# 输出不相等的行数和行号
print(f"total_computed 和 order_total 不相等的行数: {mismatch_count}")
print(f"不相等的行号: {mismatch_indices}")

# 如果你想查看这些行的具体数据，也可以打印它们
print(mismatch_rows[['total_computed', 'order_total', 'order_price', 'coupon_discount', 'delivery_charges']])

total_computed 和 order_total 不相等的行数: 53
不相等的行号: [7, 11, 13, 15, 16, 25, 27, 41, 71, 94, 103, 106, 111, 118, 126, 130, 141, 157, 159, 168, 179, 181, 186, 235, 241, 248, 256, 261, 268, 280, 288, 289, 300, 316, 327, 336, 357, 358, 370, 379, 382, 391, 395, 404, 410, 426, 448, 449, 463, 475, 481, 487, 499]
    total_computed  order_total  order_price  coupon_discount  \
7          7664.99      2844.79         7960                5   
11         5121.68     13141.68         5060                0   
13         5006.43      9858.95         6585               25   
15         5864.79      4785.29         6820               15   
16        21313.28     20713.28        21260                0   
25         3780.15     20997.77         4135               10   
27         7951.96     10581.34         9275               15   
41        31403.05     11478.48        32970                5   
71         4324.14      9602.67         4685               10   
94         9153.28      2218.78        10080   

<div class="alert alert-block alert-info">
    
### 4.6. Fix the order_price<a class="anchor" name="Reg_Exp"></a>

<div class="alert alert-block alert-info">
    
### 4.7. Fix the shopping_cart<a class="anchor" name="Reg_Exp"></a>

<div class="alert alert-block alert-info">
    
### 4.9. Fix the is_happy_customer<a class="anchor" name="Reg_Exp"></a>

<div class="alert alert-block alert-info">
    
### 4.10. Fix the is_expedited_delivery<a class="anchor" name="Reg_Exp"></a>

In [ ]:
# 获取 dirty_data 中 error 列标记为 1 的行号
error_indices = dirty_data.index[dirty_data['error'] == 1]
# 打印出这些行号
print(error_indices)

Since this business is based in Melbourne, the correct values for latitude and longitude should be Latitude: -37.8136° and Longitude: 144.9631°. However, I have discovered some incorrect data where the latitude and longitude values were swapped. After correcting these errors, the distance_to_nearest_warehouse and nearest_warehouse fields, which are calculated based on the latitude and longitude, may also need to be fixed accordingly.

In [ ]:
# Check for latitude greater than 0 (latitude in Australia should be less than 0)
lat_issue = dirty_data.loc[dirty_data['customer_lat'] > 0]
print(lat_issue[['customer_lat', 'customer_long']])

# Check for longitude less than 0 (longitude in Australia should be greater than 0)
long_issue = dirty_data.loc[dirty_data['customer_long'] < 0]
print(long_issue[['customer_lat', 'customer_long']])

# Condition to find rows where either longitude < 0 or latitude > 0
condition = (dirty_data['customer_long'] < 0) | (dirty_data['customer_lat'] > 0)

# Select the rows matching the condition and swap the latitude and longitude
#reference from chatGPT
dirty_data.loc[condition, ['customer_lat', 'customer_long']] = \
    dirty_data.loc[condition, ['customer_long', 'customer_lat']].values
    

After fixing the customer latitude and longitude, I need to recalculate the distance_to_nearest_warehouse and reassign the nearest_warehouse data for each customer.

In [40]:
# Load warehouse data
warehouse = pd.read_csv("warehouses.csv")
lat = dict(zip(warehouse['names'], warehouse['lat']))
lon = dict(zip(warehouse['names'], warehouse['lon']))

# reference from https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
# Haversine function to calculate distance between two coordinates
def haversine(lat1, lon1, lat2, lon2):
    dLat = (lat2 - lat1) * math.pi / 180.0 
    dLon = (lon2 - lon1) * math.pi / 180.0
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
    
    a = (pow(math.sin(dLat / 2), 2) + 
         pow(math.sin(dLon / 2), 2) * 
         math.cos(lat1) * math.cos(lat2))
# Earth radius in kilometers
    rad = 6378
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

# Calculate the nearest warehouse and distance for each customer
for index, row in dirty_data.iterrows():
    # Get customer latitude
    customer_lat = row['customer_lat']
    # Get customer longitude
    customer_long = row['customer_long']
    # Initialize min distance to infinity
    min_distance = float('inf')
     # Initialize nearest warehouse name
    nearest_warehouse_name = None
    # Calculate the distance to each warehouse
    for name in lat:
        dist = round(haversine(customer_lat, customer_long, lat[name], lon[name]), 4)
        # Update min distance and nearest warehouse if closer
        if dist < min_distance:
            min_distance = dist
            nearest_warehouse_name = name
    
    # Update dirty_data with the nearest warehouse and distance
    dirty_data.at[index, 'nearest_warehouse'] = nearest_warehouse_name
    dirty_data.at[index, 'distance_to_nearest_warehouse'] = min_distance

# Count the occurrences of each warehouse in the 'nearest_warehouse' column
warehouse_count = dirty_data['nearest_warehouse'].value_counts()
# Print the result
print(warehouse_count)

nearest_warehouse
Nickolson    181
Thompson     179
Bakers       140
Name: count, dtype: int64


Based on the assignment information, we know that the store sells only 10 branded items. Therefore, I need to first identify which brands are being sold and how many items are being sold for each brand.

In [32]:
# 初始化一个 Counter 来统计每个品牌的销售数量
brand_counter = Counter()

# 遍历 outlier_data 数据集以获取所有品牌
for index, row in outlier_data.iterrows():
    # 将 shopping_cart 列的字符串转换为 Python 列表
    shopping_cart = ast.literal_eval(row['shopping_cart'])
    # 遍历购物车中的每个商品
    for item in shopping_cart:
        brand_name = item[0]  # 获取商品的品牌名称
        brand_counter[brand_name] += 1  # 统计每个品牌的出现次数

# 获取所有的品牌列表
item_types = list(brand_counter.keys())
print(item_types)

# 初始化矩阵 A 和向量 b
A = np.zeros((len(outlier_data), len(item_types)))  # A 矩阵的形状是 (订单数量, 品牌数量)
b = np.zeros(len(outlier_data))  # b 是存储订单价格的向量

# 遍历 outlier_data 数据集来填充矩阵 A 和向量 b
for index, row in outlier_data.iterrows():
    # 将 shopping_cart 列的字符串形式转换为 Python 列表
    shopping_cart = ast.literal_eval(row["shopping_cart"])  # 使用 ast.literal_eval 提高安全性
    # 将订单的价格存储到向量 b 中
    b[index] = row["order_price"]
    # 遍历购物车中的每个商品和数量
    for item in shopping_cart:
        brand_name = item[0]
        quantity = item[1]
        # 如果商品属于已知的 item_types
        if brand_name in item_types:
            # 找到该商品在 item_types 列表中的索引
            item_index = item_types.index(brand_name)
            # 将商品的数量添加到矩阵 A 的相应位置
            A[index, item_index] += quantity

# 检查 A 和 b 中是否有全为零的行或无效值
valid_indices = np.where(A.any(axis=1) & ~np.isnan(b))[0]
A = A[valid_indices]
b = b[valid_indices]

# 使用 np.linalg.lstsq() 计算每个品牌的价格
prices, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

# 对价格进行四舍五入处理，不保留小数点
rounded_prices = np.round(prices)

# 将结果转换为品牌价格的字典
price_dict = dict(zip(item_types, rounded_prices))

# 输出每个品牌的价格
print("Brand Prices (Rounded):")
for brand, price in price_dict.items():
    print(f"Brand: {brand}, Price: {int(price)}")  # 转换为整数形式输出

['iAssist Line', 'Lucent 330S', 'Toshika 750', 'Thunder line', 'Olivia x460', 'Universe Note', 'iStream', 'Alcon 10', 'Candle Inferno', 'pearTV']
Brand Prices (Rounded):
Brand: iAssist Line, Price: 2225
Brand: Lucent 330S, Price: 1230
Brand: Toshika 750, Price: 4320
Brand: Thunder line, Price: 2180
Brand: Olivia x460, Price: 1225
Brand: Universe Note, Price: 3450
Brand: iStream, Price: 150
Brand: Alcon 10, Price: 8950
Brand: Candle Inferno, Price: 430
Brand: pearTV, Price: 6310


In [28]:
print("Matrix A:")
print(A)

print("Vector b:")
print(b)


Matrix A:
[[2. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 2. ... 2. 0. 0.]
 [0. 2. 2. ... 0. 0. 0.]
 [0. 0. 0. ... 2. 0. 0.]]
Vector b:
[19815.  1525.  6930. 20210.  9500. 13700. 15970.    nan  7015. 13260.
 11400.  3750. 11990. 14275.    nan 23080. 13485. 13320.    nan 23445.
 22490. 13270. 14070. 19125. 22160. 13360.  6930. 10985.  4540. 14845.
 31380. 15965.  9865. 10950. 12555.  3320. 11310. 17090.    nan 24710.
  3455. 12090. 12400. 12855.  8535. 11490. 15690. 15930.  8155. 18200.
 12415. 13580. 12130. 14980. 14355. 13610.  8810. 18320.  3455. 13000.
 23575.  4660.  3980.  3835.  1380. 37300. 21580. 15100. 13630.  9780.
  3040. 16720.  8940. 20080. 15460.  4470.  4750. 33480. 18200. 22250.
  6170. 12120. 13570.  9125.  4675. 10660. 16175. 12570.  7765.  8285.
 25710.  2390. 29150.  4510.  3310.  2375. 18050. 17760. 12015.  8680.
  6500.  4880.  3410. 13460. 23485.  7770. 35310.  6780. 15720. 10520.
 17025. 10630. 19580.  8070. 35560. 12400. 

In [24]:
A = np.zeros((len(outlier_data), len(item_types)))  # A 矩阵的形状是 (缺失数据行数, 品牌数量)
b = np.zeros(len(outlier_data))  # b 是长度为缺失数据行数的零向量

for index, row in outlier_data.iterrows():
    # 将 shopping_cart 列的字符串形式转换为 Python 列表
    shopping_cart = ast.literal_eval(row["shopping_cart"])  # 使用 ast.literal_eval 提高安全性
    # 将订单的价格存储到向量 b 中
    b[index] = row["order_price"]
    # 遍历购物车中的每个商品和数量
    for item, quantity in shopping_cart:
        # 如果商品属于已知的 item_types
        if item in item_types:
            # 找到该商品在 item_types 列表中的索引
            item_index = item_types.index(item)
            # 将商品的数量添加到矩阵 A 的相应位置
            A[index, item_index] += quantity

# 使用 np.linalg.lstsq() 计算每个品牌的价格
prices, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

# 将结果转换为品牌价格的字典
price_dict = dict(zip(item_types, prices))

# 输出每个品牌的价格
print("Brand Prices:")
for brand, price in price_dict.items():
    print(f"Brand: {brand}, Price: {price}")

[19815.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0

In [ ]:
#shopping_cart & order_price
for index, row in dirty_data.iterrows():
    
    # 将字符串形式的 shopping_cart 列转换为 Python 列表
    shopping_cart = eval(row['shopping_cart'])
    
  
item_types = set()  # 初始化一个集合来存储商品名称

for index, row in missing_data_Nickolson.iterrows():
    shopping_cart = ast.literal_eval(row['shopping_cart'])  # 解析购物车字符串
    for item in shopping_cart:
        item_types.add(item[0])  # 将商品名称添加到集合中



<div class="alert alert-block alert-warning"> 

## 5.  Detect and fix errors in dirty_data <a class="anchor" name="load"></a>

</div>

Empty DataFrame
Columns: [is_happy_customer]
Index: []
